# SQLModel Session exercise
Adapted from Drew's slack example

In [1]:
from factories import *

## Create a new Batch from factory

In [2]:
batch = BatchFactory.create()

2023-04-05 13:11:53,797 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-04-05 13:11:53,800 INFO sqlalchemy.engine.Engine INSERT INTO batches (name, description) VALUES (?, ?)
2023-04-05 13:11:53,802 INFO sqlalchemy.engine.Engine [generated in 0.00180s] ('Batch 0', 'Batch 0 Description')
2023-04-05 13:11:53,804 INFO sqlalchemy.engine.Engine COMMIT


## Grab the session the Batch was created with for inspection.

In [3]:
session = Session.object_session(batch)

## Change something on the batch

In [4]:
batch.description = "exceedingly jolly"

## View the changes that have yet to be committed to the db

In [5]:
print('session.dirty = ', session.dirty)
# => session.dirty = IdentitySet([Batch(description='foo')])

session.dirty =  IdentitySet([Batch(description='exceedingly jolly')])


## Commit the changes to the db. 
Note that we do not have to call session.add(batch) since the Batch instance is already attached to the session.

In [6]:
session.commit()

2023-04-05 13:11:54,189 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-04-05 13:11:54,193 INFO sqlalchemy.engine.Engine SELECT batches.id AS batches_id, batches.name AS batches_name 
FROM batches 
WHERE batches.id = ?
2023-04-05 13:11:54,194 INFO sqlalchemy.engine.Engine [generated in 0.00104s] (14,)
2023-04-05 13:11:54,196 INFO sqlalchemy.engine.Engine UPDATE batches SET description=? WHERE batches.id = ?
2023-04-05 13:11:54,197 INFO sqlalchemy.engine.Engine [generated in 0.00072s] ('exceedingly jolly', 14)
2023-04-05 13:11:54,198 INFO sqlalchemy.engine.Engine COMMIT


## Verify there are no other pending changes.

In [7]:
print('session.dirty = ', session.dirty)
# => IdentitySet([])

session.dirty =  IdentitySet([])



## Use a contextual session 
(called "db") to grab a separate copy of the batch and verify it has the change.

In [8]:
with Session(engine) as db:
   batch_copy = db.exec(
       select(Batch).where(Batch.id==batch.id)
   ).first()
print('batch_copy.description = ', batch_copy.description)
# => foo

2023-04-05 13:11:54,422 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-04-05 13:11:54,423 INFO sqlalchemy.engine.Engine SELECT batches.id AS batches_id, batches.name AS batches_name, batches.description AS batches_description 
FROM batches 
WHERE batches.id = ?
2023-04-05 13:11:54,424 INFO sqlalchemy.engine.Engine [generated in 0.00088s] (14,)
2023-04-05 13:11:54,426 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-04-05 13:11:54,428 INFO sqlalchemy.engine.Engine SELECT batches.id, batches.name, batches.description 
FROM batches 
WHERE batches.id = ?
2023-04-05 13:11:54,429 INFO sqlalchemy.engine.Engine [generated in 0.00082s] (14,)
2023-04-05 13:11:54,430 INFO sqlalchemy.engine.Engine ROLLBACK
batch_copy.description =  exceedingly jolly


Note that since we grabbed batch_copy using a contextual session, and that context has ended, so has the session, and the object is currently unattached to any session.

In [9]:
print('seesion for batch_copy = ', Session.object_session(batch_copy))
# => None

seesion for batch_copy =  None


## Now change batch_copy.descripiton, 
save it with a contextual session, and see how it affected or original Batch instance, which is still alive and attached to the session it was created with.

In [10]:
batch_copy.description += " and effortlessly charming"
with Session(engine) as db:
   db.add(batch_copy)
   db.commit()

2023-04-05 13:11:54,703 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-04-05 13:11:54,705 INFO sqlalchemy.engine.Engine UPDATE batches SET description=? WHERE batches.id = ?
2023-04-05 13:11:54,706 INFO sqlalchemy.engine.Engine [cached since 0.5094s ago] ('exceedingly jolly and effortlessly charming', 14)
2023-04-05 13:11:54,707 INFO sqlalchemy.engine.Engine COMMIT


## NOTE: after committing changes to batch_copy using contextual session in this way,
trying to access attributes on the object like batch_copy.description raises an error: 
> Instance <Batch at 0x107b84e10> is not bound to a Session; attribute refresh operation cannot proceed

In [11]:
with Session(engine) as db:
   db.add(batch_copy)
   print('batch_copy.description = ', batch_copy.description)
   # => batch_copy.description =  exceedingly jolly and effortlessly charming

2023-04-05 13:11:54,955 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-04-05 13:11:54,956 INFO sqlalchemy.engine.Engine SELECT batches.id AS batches_id, batches.name AS batches_name, batches.description AS batches_description 
FROM batches 
WHERE batches.id = ?
2023-04-05 13:11:54,957 INFO sqlalchemy.engine.Engine [cached since 0.534s ago] (14,)
batch_copy.description =  exceedingly jolly and effortlessly charming
2023-04-05 13:11:54,959 INFO sqlalchemy.engine.Engine ROLLBACK


## Confirm that our original Batch instance still has the old description

In [12]:
print('batch.description before refresh = ', batch.description)
# => batch.description before refresh = exceedingly jolly


batch.description before refresh =  exceedingly jolly


## Refresh the original Batch instance
and check the description again

In [13]:
session.refresh(batch)
print('batch.description after refresh = ', batch.description)
# => batch.description after refresh = exceedingly jolly and effortlessly charming


2023-04-05 13:11:55,366 INFO sqlalchemy.engine.Engine SELECT batches.id, batches.name, batches.description 
FROM batches 
WHERE batches.id = ?
2023-04-05 13:11:55,368 INFO sqlalchemy.engine.Engine [generated in 0.00136s] (14,)
batch.description after refresh =  exceedingly jolly and effortlessly charming


## Now let's edit `batch.description` and `batch_copy.description`
and update them both independently to see what happens.

In [14]:
batch.description += " and fantastically flatulent"


## save the first edit

In [15]:

session.commit()

2023-04-05 13:11:55,512 INFO sqlalchemy.engine.Engine UPDATE batches SET description=? WHERE batches.id = ?
2023-04-05 13:11:55,513 INFO sqlalchemy.engine.Engine [cached since 1.317s ago] ('exceedingly jolly and effortlessly charming and fantastically flatulent', 14)
2023-04-05 13:11:55,515 INFO sqlalchemy.engine.Engine COMMIT


## save the second edit


In [16]:
with Session(engine) as db:
   batch_copy.description += " and elaborately loquacious"
   db.add(batch_copy)
   db.commit()
   print('batch_copy.description = ', batch_copy.description)
# => batch_copy.description =  exceedingly jolly and effortlessly charming and elaborately loquacious

2023-04-05 13:11:55,579 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-04-05 13:11:55,580 INFO sqlalchemy.engine.Engine UPDATE batches SET description=? WHERE batches.id = ?
2023-04-05 13:11:55,581 INFO sqlalchemy.engine.Engine [cached since 1.385s ago] ('exceedingly jolly and effortlessly charming and elaborately loquacious', 14)
2023-04-05 13:11:55,583 INFO sqlalchemy.engine.Engine COMMIT
2023-04-05 13:11:55,586 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-04-05 13:11:55,586 INFO sqlalchemy.engine.Engine SELECT batches.id AS batches_id, batches.name AS batches_name, batches.description AS batches_description 
FROM batches 
WHERE batches.id = ?
2023-04-05 13:11:55,587 INFO sqlalchemy.engine.Engine [cached since 1.164s ago] (14,)
batch_copy.description =  exceedingly jolly and effortlessly charming and elaborately loquacious
2023-04-05 13:11:55,589 INFO sqlalchemy.engine.Engine ROLLBACK


## Unsurprisingly, the 2nd save wins. 
The changes saved in original Batch instance with it's original session get overwritten by this one.